In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
ss = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

X = train_df.drop(['target', 'id'], axis = 1).values
y = train_df['target'].values
X_test = test_df.drop('id', axis = 1).values

del train_df, test_df

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import FastICA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

X = StandardScaler().fit_transform(X)

pca = FastICA()
comp_X = pca.fit_transform(X)

cs = np.logspace(-5, 4, 30)

regr = LogisticRegression(penalty = 'l1', solver = 'saga', warm_start = True, max_iter = 10_000)
pca_regr = LogisticRegression(penalty = 'l1', solver = 'saga', warm_start = True, max_iter = 10_000)

In [ ]:
from tqdm.notebook import tqdm

feature_names = [f"f{i+1}" for i in range(100)]
pc_names = [f"PC{i+1}" for i in range(100)]

pca_coef = pca.components_.copy()

regr_coef = []
pca_regr_coef = []
for c in tqdm(cs):
    regr.set_params(C = c)
    regr.fit(X, y)
    regr_coef.append(regr.coef_.ravel().copy())
    
    pca_regr.set_params(C = c)
    pca_regr.fit(comp_X, y)
    pca_regr_coef.append(pca_regr.coef_.ravel().copy())

In [ ]:
plain_regr_imp = np.array(regr_coef)
pca_regr_imp = np.array(pca_regr_coef).dot(pca_coef)

In [ ]:
fig, axs = plt.subplots(25, 4, figsize = (15, 60))

_max = max(plain_regr_imp.max(), pca_regr_imp.max())
_min = min(plain_regr_imp.min(), pca_regr_imp.min())

for i, ax in enumerate(axs.ravel()):
    sns.lineplot(x = np.log10(cs), y = plain_regr_imp[:,i], marker = "o", ax = ax)
    sns.lineplot(x = np.log10(cs), y = pca_regr_imp[:,i], marker = "X", ax = ax)
    ax.axhline(0)
    ax.set(
#         ylim = (_min, _max),
        title = f"f{i}"
    )
    

plt.tight_layout()
plt.show()

# Orange with X is importance via component
# Blue with solid O is importance on its own